<span style="color:#8B949E;">
<b>Note de lecture</b> — Notebook issu de tests itératifs (“speed-tests”).  
Le corpus utilisé ici est un <b>dataset de substitution</b> (non aligné client) uniquement pour valider la mécanique (retrieval + évaluation) et dérouler la roadmap.  
Pour le chemin complet, suivre l’ordre 01 → 10 et lire en priorité les sections Markdown.
</span>


# 🔎 BM25 simple sur corpus XML Légifrance (POC)

> **But** : démarrer par un **BM25 pur** sur des articles réels pour **mesurer** (Recall@k, qualité du ranking) et **repérer** les faux négatifs / limites du lexical **avant** d’ajouter une couche sémantique (embeddings/LLM).

---

## 🧭 Plan du notebook
1. 🧩 Étape 1 — Parcourir et charger les articles XML  
2. 🧩 Étape 2 — Construire un index **BM25** (rank-bm25)  
3. 🧩 Étape 3 — Interroger le moteur (requête + top-k)  
4. ✅ Ce qu’on observe (forces / limites)

💡 **Principe volontairement simple** : *1 fichier XML = 1 article* (hypothèse POC, assumée).


## ⚙️ Pré-requis & configuration

### 📦 Dépendance Python
Ce notebook utilise `rank-bm25` :

```bash
pip install rank-bm25
```

### 🗂️ Chemin du corpus
Adapte `DATA_ROOT` vers le dossier racine qui contient tes fichiers **.xml**.


In [1]:
# --- Configuration ---
# Adapte ce chemin à notre machine / notre arborescence.
# Exemple Windows :
DATA_ROOT = r"D:\-- Projet RAG Avocats --\data_main\data"
# Exemple Linux / WSL :
# DATA_ROOT = r"/mnt/d/-- Projet RAG Avocats --/data_main/data"

## 🧩 Étape 1 — Parcourir et charger les articles XML

🎯 Objectif :
- parcourir tous les `.xml` sous `DATA_ROOT`
- extraire **tout le texte** (version simple & robuste)
- construire un identifiant stable (**doc_id**)  
- ignorer le reste (pour l’instant)

👉 On ne cherche pas encore la perfection juridique : on veut une **base saine** pour évaluer le lexical.


In [2]:
import os
import xml.etree.ElementTree as ET
from typing import Optional

def extract_text_from_xml(xml_path: str) -> Optional[str]:
    """
    Extrait tout le texte contenu dans un fichier XML.
    Version volontairement simple et robuste :
    - parse le XML
    - récupère tous les textes des nœuds
    - concatène en une seule chaîne
    """
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()

        texts = []
        for elem in root.iter():
            if elem.text:
                t = elem.text.strip()
                if t:
                    texts.append(t)

        return " ".join(texts)
    except Exception:
        # On reste volontairement silencieux pour éviter de casser le batch sur un XML abîmé
        return None


documents = []

for root_dir, _, files in os.walk(DATA_ROOT):
    for file in files:
        if file.lower().endswith(".xml"):
            full_path = os.path.join(root_dir, file)
            text = extract_text_from_xml(full_path)

            # Filtre simple pour éviter les fichiers trop petits / bruités
            if text and len(text) > 200:
                documents.append({
                    "doc_id": full_path,
                    "text": text
                })

print(f"Nombre total de documents chargés : {len(documents)}")

Nombre total de documents chargés : 4395


## 🧩 Étape 2 — Construire un index BM25

🎯 Objectif : créer un moteur de recherche lexical (conceptuellement proche d’un Elasticsearch en mode BM25)

- tokenisation volontairement simple (`lower().split()`)
- construction de l’index BM25 via `BM25Okapi`


In [3]:
from rank_bm25 import BM25Okapi

def tokenize(text: str) -> list[str]:
    """
    Tokenisation minimaliste pour POC :
    - passe en minuscules
    - split sur les espaces
    """
    return text.lower().split()

corpus_tokens = [tokenize(doc["text"]) for doc in documents]
bm25 = BM25Okapi(corpus_tokens)

print("Index BM25 prêt ✅")

Index BM25 prêt ✅


## 🧩 Étape 3 — Interroger le BM25

🎯 Objectif :
- poser une question juridique
- récupérer un **top-k** de documents
- inspecter rapidement les retours (début du texte + score)

🔍 À observer :
- BM25 favorise les **termes exacts**
- peut rater les synonymes / paraphrases
- on veut justement voir le **bruit** (FP) et les **manques** (FN)


In [4]:
def bm25_search(query: str, top_k: int = 5):
    """
    Recherche BM25 :
    - score tous les documents
    - trie par score décroissant
    - renvoie les top_k (document, score)
    """
    query_tokens = tokenize(query)
    scores = bm25.get_scores(query_tokens)

    ranked = sorted(
        zip(documents, scores),
        key=lambda x: x[1],
        reverse=True
    )
    return ranked[:top_k]


query = "rupture du CDI sans préavis"
results = bm25_search(query, top_k=5)

for rank, (doc, score) in enumerate(results, start=1):
    print(f"\n--- Résultat {rank} | Score: {score:.2f}")
    print(f"doc_id: {doc['doc_id']}")
    print(doc["text"][:500])


--- Résultat 1 | Score: 18.98
doc_id: D:\-- Projet RAG Avocats --\data_main\data\20250731-220719\legi\global\code_et_TNC_en_vigueur\code_en_vigueur\LEGI\TEXT\00\00\06\07\20\LEGITEXT000006072050\article\LEGI\ARTI\00\00\21\92\56\LEGIARTI000021925677.xml
LEGIARTI000021925677 LEGI article/LEGI/ARTI/00/00/21/92/56/LEGIARTI000021925677.xml Article L3142-17 MODIFIE 2010-03-04 2016-08-10 AUTONOME Code du travail Partie législative Troisième partie : Durée du travail, salaire, intéressement, participation et épargne salariale Livre Ier : Durée du travail, repos et congés Titre IV : Congés payés et autres congés Chapitre II : Autres congés Section 2 : Congés non rémunérés Section 2 : Congés pour engagement associatif, politique ou militant Sous-section

--- Résultat 2 | Score: 13.76
doc_id: D:\-- Projet RAG Avocats --\data_main\data\20250731-220719\legi\global\code_et_TNC_en_vigueur\TNC_en_vigueur\JORF\TEXT\00\00\20\23\71\JORFTEXT000020237165\article\LEGI\ARTI\00\00\51\86\72\LEGIARTI00005186728

## ✅ Lecture des résultats : forces & limites attendues

### 👍 Ce que BM25 fait bien
- Très bon sur les **mots clés exacts**
- Très explicable (scores / matching)

### ⚠️ Ce que BM25 fait moins bien (normal)
- Synonymes / reformulations (ex : *licenciement* vs *rupture*…)
- Variantes morphologiques / accents / ponctuation
- Dépend fortement de la tokenisation (ici, volontairement minimaliste)

### 🧠 Pourquoi cette étape est utile
Avant d’introduire embeddings/LLM, on veut **mesurer** et **comprendre** :
- où se trouvent les faux négatifs,
- ce que le lexical “rate” intrinsèquement,
- et comment le ranking se comporte sur des requêtes réalistes.
